In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!pip3 install -r requirements.txt
!python3 setup.py develop

In [ ]:
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

In [5]:
! pip install -q kaggle

In [10]:
!pip3 install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from google.colab import files
files.upload()         # expire any previous token(s) and upload recreated token

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tuanluchuynh","key":"f71f90061b7fa423897f246238ec2176"}'}

In [8]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d vlbthambawita/visemtracking

 99% 1.76G/1.78G [00:48<00:00, 48.3MB/s]
100% 1.78G/1.78G [00:48<00:00, 39.5MB/s]


In [ ]:
!unzip /content/ByteTrack/visemtracking.zip

In [11]:
import os
import numpy as np
import json
import cv2
from natsort import natsorted

In [14]:
IMAGE_PATH = '/content/ByteTrack/VISEM_Tracking_Train_v4/Train'
num = np.sort(np.array(os.listdir(IMAGE_PATH), dtype='int32'))
from sklearn.model_selection import train_test_split
train_num, val_num = train_test_split(num, test_size=0.2, random_state=42)

In [16]:
### CREATE A VAL FOLDER AND MOVE
from pathlib import Path
Path("/content/ByteTrack/VISEM_Tracking_Train_v4/Val").mkdir(parents=True, exist_ok=True)
Path("/content/ByteTrack/VISEM_Tracking_Train_v4/annotations").mkdir(parents=True, exist_ok=True)
for folder in val_num:
    source = os.path.join(IMAGE_PATH, str(folder))
    dest = "/content/ByteTrack/VISEM_Tracking_Train_v4/Val"
    !mv $source $dest

In [21]:
# Use the same script for MOT16
DATA_PATH = '/content/ByteTrack/VISEM_Tracking_Train_v4/'
OUT_PATH = os.path.join(DATA_PATH, 'annotations')
SPLITS = ['Train', 'Val']


if __name__ == '__main__':

    if not os.path.exists(OUT_PATH):
        os.makedirs(OUT_PATH)

    for split in SPLITS:
        if split == "Val":
            data_path = os.path.join(DATA_PATH, 'Val')
        else:
            data_path = os.path.join(DATA_PATH, 'Train')
        out_path = os.path.join(OUT_PATH, '{}.json'.format(split))
        out = {'images': [], 'annotations': [], 'videos': [],
               'categories': [{'id': 0, 'name': 'sperm'},
                              {'id': 1, 'name': 'cluster'},
                              {'id': 2, 'name': 'small or pinhead'}]}

        seqs = os.listdir(data_path)
        image_cnt = 0
        ann_cnt = 0
        video_cnt = 0

        for seq in natsorted(seqs):
            if seq.endswith('.ipynb_checkpoints'):
                continue
            video_cnt += 1  # video sequence number.
            out['videos'].append({'id': video_cnt, 'file_name': seq})
            seq_path = os.path.join(data_path, seq)
            img_path = os.path.join(seq_path, 'images')
            ann_path = os.path.join(seq_path, 'labels')
            images = os.listdir(img_path)
            num_images = len([image for image in images if 'jpg' in image])  # half and half
            image_range = [0, num_images - 1]
            print('{}: {} images'.format(seq, num_images))
            
            for i, img_name in enumerate(natsorted(images)):
                if i < image_range[0] or i > image_range[1]:
                    continue
                # Image
                img = cv2.imread(os.path.join(
                    img_path, img_name))
                print('img path:', os.path.join(
                    img_path, img_name))
                height, width = img.shape[:2]
                image_info = {'file_name': img_name,  # image name.
                              'id': image_cnt + i + 1,  # image number in the entire training set.
                              # image number in the video sequence, starting from 1.
                              'frame_id': i + 1 - image_range[0],
                              # image number in the entire training set.
                              'prev_image_id': image_cnt + i if i > 0 else -1,
                              'next_image_id': image_cnt + i + 2 if i < num_images - 1 else -1,
                              'video_id': seq,
                              'height': height, 'width': width}
                out['images'].append(image_info)
                # Label
                annotations_path = os.path.join(ann_path, '{}txt'.format(img_name[:-3]))
                if os.path.exists(annotations_path) == True:
                    anns = np.loadtxt(annotations_path,
                                    dtype=np.float64, delimiter=' ')
                    anns = np.atleast_2d(anns)
                    print('label path:', annotations_path)
                    for j in range(anns.shape[0]):
                        category_id = int(anns[j][0])
                        frame_id = j + 1 - image_range[0]
                        if frame_id - 1 < image_range[0] or frame_id - 1 > image_range[1]:
                            continue
                        ann_cnt += 1
                        ann = {'id': ann_cnt,
                            'category_id': category_id,
                            'image_id': image_cnt + i + 1,
                            'bbox': anns[j][1:].tolist()}
                    #     out['annotations'].append(ann)
                    # print('{}_frame_{}: {} annotations'.format(seq, i, anns.shape[0]))
                else:
                    
                    ann = {'id': ann_cnt,
                            'category_id': category_id,
                            'image_id': image_cnt + i + 1,
                            'bbox': []}
                out['annotations'].append(ann)
                print('{}_frame_{}: {} annotations'.format(seq, i, len(ann['bbox'])))
            image_cnt += num_images
        print('loaded {} for {} images and {} annotations'.format(
            split, len(out['images']), len(out['annotations'])))
        json.dump(out, open(out_path, 'w'))


Streaming output truncated to the last 5000 lines.
img path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/images/47_frame_1274.jpg
label path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/labels/47_frame_1274.txt
47_frame_1274: 4 annotations
img path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/images/47_frame_1275.jpg
label path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/labels/47_frame_1275.txt
47_frame_1275: 4 annotations
img path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/images/47_frame_1276.jpg
label path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/labels/47_frame_1276.txt
47_frame_1276: 4 annotations
img path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/images/47_frame_1277.jpg
label path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/labels/47_frame_1277.txt
47_frame_1277: 4 annotations
img path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/47/images/47_frame_1278.jpg
label path: /content/ByteTrack/VISEM_Tracking_Train_v4/Val/4

In [23]:
!python3 tools/train.py -f '/content/ByteTrack/exps/example/mot/visem.py' -d 8 -b 48 --fp16 -o -c pretrained/yolox_x.pth

Traceback (most recent call last):
  File "tools/train.py", line 112, in <module>
    assert num_gpu <= torch.cuda.device_count()
AssertionError


In [22]:
!cp '/content/ByteTrack/exps/example/mot/yolox_x_ch.py' '/content/ByteTrack/exps/example/mot/visem.py'
